In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from config import gkey
from datetime import date
# Configure gmaps
gmaps.configure(api_key=gkey)
from pprint import pprint
import itertools



In [2]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df.head()

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json


In [3]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [4]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("bird", case=False)]
len(data_scoot)
data_scoot


,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
339,US,Bird Alexandria,"Alexandria, VA",bird-alexandria,https://www.bird.co,https://mds.bird.co/gbfs/alexandria/gbfs.json
340,US,Bird Chicago,"Chicago, IL",bird-chicago,https://www.bird.co,https://mds.bird.co/gbfs/chicago/gbfs.json
341,US,Bird Cleveland,"Cleveland, OH",bird-cleveland,https://www.bird.co,https://mds.bird.co/gbfs/cleveland/gbfs.json
342,US,Bird Columbus,"Columbus, OH",bird-columbus,https://www.bird.co,https://mds.bird.co/gbfs/columbus/gbfs.json
343,US,Bird Culver City,"Culver City, CA",bird-culver-city,https://www.bird.co,https://mds.bird.co/gbfs/culver-city/gbfs.json
344,US,Bird Detriot,"Detroit, MI",bird-detroit,https://www.bird.co,https://mds.bird.co/gbfs/detroit/gbfs.json
345,US,Bird Fairfax,"Fairfax, VA",bird-fairfax,https://www.bird.co,https://mds.bird.co/gbfs/fairfax/gbfs.json
346,US,Bird Indianapolis,"Indianapolis,IN",bird-indianapolis,https://www.bird.co,https://mds.bird.co/gbfs/indianapolis/gbfs.json
347,US,Bird Kansas City,"Kansas City, MO",bird-kansas-city,https://www.bird.co,https://mds.bird.co/gbfs/kansas-city/gbfs.json
348,US,Bird Los Angeles,"Los Angeles, CA",bird-los-angeles,https://www.bird.co,https://mds.bird.co/gbfs/los-angeles/gbfs.json


In [19]:
lat_list = []
lon_list = []
base_url= "https://mds.bird.co/gbfs/washington-dc/gbfs.json"
city = "Washington DC"
data = requests.get(base_url).json()
base_url1=data["data"]["en"]["feeds"][2]["url"]
result1 = requests.get(base_url1).json()
bike_coord = result1['data']['bikes']
print(f'There are {len(bike_coord)} bikes in {city}.')
for bike in bike_coord:
        
        # Add Lat info to list
        lat_list.append([bike][0]['lat'])
        lon_list.append([bike][0]['lon'])

There are 677 bikes in Washington DC.


In [20]:
city_bird_df = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list})
city_bird_df['Operator'] = "Bird"
today = date.today()
city_bird_df['Date'] = today
city_bird_df.to_csv('WashingtonDC_Bird_Lat_Lon.csv')

In [9]:
geocleaned_df = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list})
geocleaned_df['Operator'] = "Bird"
today = date.today()
geocleaned_df['Date'] = today
geocleaned_df
# geocleaned_df.to_csv('Geocode_Bird_Lat_Lon.csv')

,Lat,Lon,Operator,Date
0,38.994739,-77.041154,Bird,2021-05-17
1,38.994739,-77.041154,Bird,2021-05-17
2,38.893812,-76.909382,Bird,2021-05-17
3,38.799691,-77.028859,Bird,2021-05-17
4,38.827513,-77.030919,Bird,2021-05-17
5,38.854518,-77.021445,Bird,2021-05-17
6,38.871379,-77.040773,Bird,2021-05-17
7,38.875434,-77.045917,Bird,2021-05-17
8,38.885538,-77.056881,Bird,2021-05-17
9,38.888427,-77.060174,Bird,2021-05-17


In [13]:


locations = list(zip(lat_list,lon_list))
locations



[(38.8410750557452, -77.0462917608619),
 (38.8404843680438, -77.0462132610711),
 (38.8404245554607, -77.0460353857972),
 (38.840014430543, -77.0458505729493),
 (38.8399816806208, -77.0460900103779),
 (38.8400817432697, -77.0464075109622),
 (38.8398865554896, -77.046506260789),
 (38.8396533679068, -77.0463002607689),
 (38.8395464305486, -77.0463094483482),
 (38.8389936178968, -77.0459098858054),
 (38.8387387425184, -77.0453988849437),
 (38.8384763052012, -77.0452707602149),
 (38.8381488047956, -77.0454354475582),
 (38.837797054927, -77.0454180106066),
 (38.8377273676733, -77.0453316357469),
 (38.8372303051062, -77.0453275730008),
 (38.8365146175192, -77.0451981354177),
 (38.8360828670702, -77.0450313852634),
 (38.8359139924492, -77.0451471350874),
 (38.8357794923006, -77.0450410726087),
 (38.8354234919065, -77.0450096349421),
 (38.8344794918108, -77.0447303226027),
 (38.8338003045673, -77.0444351348313),
 (38.833530616655, -77.0441737595344),
 (38.8335227418627, -77.0439773222803),
 (38

In [15]:

#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating = False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))